# https://medium.com/data-science-everywhere/data-preprocessing-a-practical-guide-1b1ce3e884d8


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
training_set = pd.read_csv('./titanic/train.csv')

In [5]:
training_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
training_set.shape

(891, 12)

In [7]:
training_set.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [8]:
#  returns a number of null values in each column.
training_set.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### Preparing the dataset

In [9]:
training_set.drop(['Ticket', 'PassengerId'], 1, inplace=True)

In [10]:
training_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(4)
memory usage: 69.7+ KB


In [11]:
# user defined function 
def assignDeckValue(CabinCode):
    if pd.isnull(CabinCode):
        category = 'Unknown'
    else:
        category = CabinCode[0]
    return category
  
Deck = np.array([assignDeckValue(cabin) for cabin in training_set['Cabin'].values])

training_set = training_set.assign(Deck = Deck)

In [13]:
training_set['FamilySize'] = training_set['SibSp'] + training_set['Parch'] +1

In [14]:
# Using expression pattern to extract the Title of the passenger
training_set['Title'] = training_set.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

# Changing to common category
training_set['Title'] = training_set['Title'].replace(['Dr', 'Rev', 'Col', 'Major', 'Countess', 'Sir', 'Jonkheer', 'Lady', 'Capt', 'Don'], 'Others')
training_set['Title'] = training_set['Title'].replace('Ms', 'Miss')
training_set['Title'] = training_set['Title'].replace('Mme', 'Mrs')
training_set['Title'] = training_set['Title'].replace('Mlle', 'Miss')

In [15]:
training_set.drop(['Cabin', 'Name'], 1, inplace = True)

In [16]:
training_set.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck,FamilySize,Title
0,0,3,male,22.0,1,0,7.2500,S,Unknown,2,Mr
1,1,1,female,38.0,1,0,71.2833,C,C,2,Mrs
2,1,3,female,26.0,0,0,7.9250,S,Unknown,1,Miss
3,1,1,female,35.0,1,0,53.1000,S,C,2,Mrs
4,0,3,male,35.0,0,0,8.0500,S,Unknown,1,Mr


In [17]:
# Returns count for each category
training_set['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [18]:
# Fills null values with 'S'-most common occurence
common = 'S'
training_set['Embarked']=training_set['Embarked'].fillna('S')

In [19]:
# Checking the no of null values now
training_set['Embarked'].isnull().sum()

0

In [20]:
means = training_set.groupby('Title')['Age'].mean()

In [21]:
title_list = ['Master','Miss','Mr','Mrs','Others']
def age_missing_replace(means, dframe, title_list):
    for title in title_list:
        temp = dframe['Title'] == title 
        dframe.loc[temp, 'Age'] = dframe.loc[temp, 'Age'].fillna(means[title]) 
        
age_missing_replace(means, training_set, title_list)

In [22]:
# mapping categorical values to numerical values using map()

training_set['Embarked'] = training_set['Embarked'].map({'C':0, 'Q':1, 'S':2})
training_set['Sex'] = training_set['Sex'].map({'male':0, 'female':1})
training_set['Title'] = training_set['Title'].map({'Master':0,'Miss':1,'Mr':2,'Mrs':3,'Others':4})

In [23]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
training_set['Deck'] = le.fit_transform(training_set['Deck'])

In [24]:
training_set.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck,FamilySize,Title
0,0,3,0,22.0,1,0,7.2500,2,8,2,2
1,1,1,1,38.0,1,0,71.2833,0,2,2,3
2,1,3,1,26.0,0,0,7.9250,2,8,1,1
3,1,1,1,35.0,1,0,53.1000,2,2,2,3
4,0,3,0,35.0,0,0,8.0500,2,8,1,2


In [25]:
training_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
Survived      891 non-null int64
Pclass        891 non-null int64
Sex           891 non-null int64
Age           891 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Fare          891 non-null float64
Embarked      891 non-null int64
Deck          891 non-null int64
FamilySize    891 non-null int64
Title         891 non-null int64
dtypes: float64(2), int64(9)
memory usage: 76.6 KB
